In [ ]:
import os
from skimage import transform
import pandas as pd
import numpy as np

# Goal

Used the matched landmarks identified on slide scanner IF images and on the panoramas acquired on the CyTOF to transform the regions of interests (ROIs) defined on the IF images.
The tranformed ROIs can then directly be imported in the CyTOF software for Imaging Mass Cytometry measurements.

## Parameters
***Modify if needed***  
Enter the input directory.  
Enter a single case ID and a single panel. This script should be run once for every slide.

In [ ]:
input_dir = "/home/ubuntu/Data3/acquisition/Batch4/"

caseID = "6000"
panel = "Islet"

## Transform the landmarks
### Input settings

In [ ]:
# Input files and folders
case_dir = os.path.join(input_dir, caseID)
fn_boxes = os.path.join(case_dir, caseID + '_Boxes_' + panel + '.csv')
fn_coordinates = os.path.join(case_dir, caseID + '_Coordinates_' + panel + '.csv')
fn_output = os.path.join(case_dir, caseID + '_IMCBoxes_' + panel + '.csv')

# Column names of the Coordinates.csv and Boxes.csv files
slide_xy = ('SlideX', 'SlideY')
imc_xy = ('IMCX', 'IMCY')
xy = ['X', 'Y']
w = 'W'
h = 'H'
boxnb = ['AcqOrder']

### Read in the csv files
The landmark coordinates for the current slide are shown.

In [ ]:
dat_boxes = pd.read_csv(fn_boxes)
dat_coord = pd.read_csv(fn_coordinates)
dat_coord.drop(dat_coord.columns[0], axis = 1, inplace = True)
dat_coord

### Mirror the coordinates
In case the image orientation is inverted.
Should not be needed in the context of the region selection workflow.

In [ ]:
# dat_coord[slide_xy[0]] = -dat_coord[slide_xy[0]]
# dat_boxes[xy[0]] = -dat_boxes[xy[0]]
# dat_boxes[w] = -dat_boxes[w]

### Learn the transformation

In [ ]:
fl_glob = np.array(dat_coord.loc[:, slide_xy])
imc_glob = np.array(dat_coord.loc[:, imc_xy])
if2imc = transform.estimate_transform('projective', fl_glob, imc_glob)
xout = if2imc(fl_glob)

### Quality control
The transformed coordinates are displayed in a plot. The green and red dot should overlap almost completely (ideally, the green dots are invisible). If the dots do not overlap, recheck the landmark coordinates entered in the Coordinates.csv files and correct the potential mistakes.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
fig, ax = plt.subplots()
#plt.scatter(fl_glob[:,0], fl_glob[:,1],)
plt.scatter(imc_glob[:,0], imc_glob[:,1],color='green')
plt.scatter(xout[:,0], xout[:,1],color='red')

for i in range(imc_glob.shape[0]):
    #ax.annotate(str(i), (fl_glob[i,0],fl_glob[i,1]))
    ax.annotate(str(i), (imc_glob[i,0]-100,imc_glob[i,1]-100))
    ax.annotate(str(i), (xout[i,0],xout[i,1]))

## Transform the ROIs
The learned transformation is applied to the selected regions of interest.

In [ ]:
box_edges = list()
box_edges.append(dat_boxes.loc[:,boxnb + xy])

tdat = dat_boxes.copy()
tdat[xy[0]] = dat_boxes[xy[0]] + dat_boxes[w]
box_edges.append(tdat.loc[:,boxnb + xy])
tdat[xy[1]] = dat_boxes[xy[1]] -dat_boxes[h]
box_edges.append(tdat.loc[:,boxnb + xy])
tdat = dat_boxes.copy()
tdat[xy[1]] = dat_boxes[xy[1]] -dat_boxes[h]
box_edges.append(tdat.loc[:,boxnb + xy])

all_boxpoints = pd.concat(box_edges)

### Display the original ROI coordinates

In [ ]:
all_boxpoints

### Apply the transformation and display the transformed coordinates

In [ ]:
all_boxpoints.loc[:, xy] = if2imc(all_boxpoints.loc[:, xy])
all_boxpoints.sort_values(boxnb)

In [ ]:
# Dispaly the unformatted transformed ROI coordinates
# all_boxpoints.groupby(boxnb).agg(['min', 'max', lambda x: np.max(x)-np.min(x)])[:]

In [ ]:
imcboxes = [all_boxpoints.groupby(boxnb).agg(['min']),
            all_boxpoints.groupby(boxnb).agg([lambda x: np.max(x)-np.min(x)])]
imcboxes = pd.concat(imcboxes, axis=1)
imcboxes = imcboxes.round(0)
imcboxes.columns = ['X', 'Y', 'W', 'H']
imcboxes

## Export the transformed ROI coordinates
The generated `.csv` file can directly be imported in the CyTOF software.

In [ ]:
imcboxes.to_csv(fn_output)